<h1 style="color:#ffc0cb;font-size:70px;font-family:Georgia;text-align:center;"><strong>FIFA World Cup Matches</strong></h1>

<a id="1"></a>
<h1 style="color:#ffc0cb;font-size:40px;font-family:Georgia;text-align:center;"><strong>1. Data Preparation</strong></h1>

<a id="1.1"></a>
# 1.1 Importing Necessary Libraries and datasets

In [1]:
# Install a conda package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install missingno


# work with data in tabular representation
from datetime import time
import pandas as pd
# round the data in the correlation matrix
import numpy as np
import os


# Modules for data visualization
import seaborn as sns
import missingno as msno

# ignore DeprecationWarning Error Messages
import warnings
warnings.filterwarnings('ignore')

In [2]:
# check the version of the packages
print("Numpy version: ", np.__version__)
print("Pandas version: ",pd.__version__)
! python --version

Numpy version:  1.20.3
Pandas version:  1.3.4
Python 3.9.7


<a id="1.2"></a>
# Data Retrieving
***
In order to load data properly, the data in csv file have to be examined carefully. First of all, all the categories are seperated by the "," and strip the extra-whitespaces at the begin by setting "skipinitialspace = True".

In [3]:
# set the path of the external data from the third party source - Kaggle
external_data_path = os.path.join(os.path.pardir, '', 'data','external')
WorldCupMatches = os.path.join(external_data_path, 'WorldCupMatches.csv')

# import dataset
WorldCupMatches = pd.read_csv(WorldCupMatches, delimiter=',', skipinitialspace = True)

print("The shape of the MATCHES data is (row, column):", str(WorldCupMatches.shape))
WorldCupMatches.head(3)

The shape of the MATCHES data is (row, column): (4572, 20)


,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA


In [4]:
# set the path of the external data from the third party source - Kaggle
external_data_path = os.path.join(os.path.pardir, '', 'data','external')
WorldCups = os.path.join(external_data_path, 'WorldCups.csv')

# import dataset
WorldCups = pd.read_csv(WorldCups, delimiter=',', skipinitialspace = True)
print("The shape of the WHOLE WORLD CUP data is (row, column):", str(WorldCups.shape))
WorldCups.head(3)

The shape of the WHOLE WORLD CUP data is (row, column): (20, 10)


,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590.549
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363.000
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375.700


<a id="1.3"></a>
## Rename column
***
Since these columns are not in SQL naming convention will be more straightforward in later process. We will also analyse the column meaning

In [5]:
print(f'The list of WorldCupMatches columns\' names is: {WorldCupMatches.columns.to_list()}; \n\nThe list of WorldCups columns\' names is: {WorldCups.columns.to_list()}\n\n\n')

The list of WorldCupMatches columns' names is: ['Year', 'Datetime', 'Stage', 'Stadium', 'City', 'Home Team Name', 'Home Team Goals', 'Away Team Goals', 'Away Team Name', 'Win conditions', 'Attendance', 'Half-time Home Goals', 'Half-time Away Goals', 'Referee', 'Assistant 1', 'Assistant 2', 'RoundID', 'MatchID', 'Home Team Initials', 'Away Team Initials']; 

The list of WorldCups columns' names is: ['Year', 'Country', 'Winner', 'Runners-Up', 'Third', 'Fourth', 'GoalsScored', 'QualifiedTeams', 'MatchesPlayed', 'Attendance']





In [6]:
WorldCupMatches.columns = ['year', 'date_time', 'stage', 'stadium', 'city', 'home_team_name',
       'home_team_goals', 'away_team_goals', 'away_team_name',
       'win_conditions', 'attendance', 'half_time_home_goals',
       'half_time_away_goals', 'referee', 'assistant_1', 'assistant_2',
       'round_id', 'match_id', 'home_team_initials', 'away_team_initials']

WorldCups.columns = ['year', 'country', 'winner', 'runners_up', 'third', 'fourth', 'goals_scored', 'qualified_teams', 'matches_played', 'attendance']

In [7]:
print(f'The list of WorldCupMatches columns\' names is: {WorldCupMatches.columns.to_list()}; \n\nThe list of WorldCups columns\' names is: {WorldCups.columns.to_list()}\n\n\n')

The list of WorldCupMatches columns' names is: ['year', 'date_time', 'stage', 'stadium', 'city', 'home_team_name', 'home_team_goals', 'away_team_goals', 'away_team_name', 'win_conditions', 'attendance', 'half_time_home_goals', 'half_time_away_goals', 'referee', 'assistant_1', 'assistant_2', 'round_id', 'match_id', 'home_team_initials', 'away_team_initials']; 

The list of WorldCups columns' names is: ['year', 'country', 'winner', 'runners_up', 'third', 'fourth', 'goals_scored', 'qualified_teams', 'matches_played', 'attendance']





<a id="2"></a>
<h1 style="color:#ffc0cb;font-size:40px;font-family:Georgia;text-align:center;"><strong>2. Data Cleaning</strong></h1>

<a id="2.2"></a>
## Data types
***

In [8]:
WorldCupMatches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572 entries, 0 to 4571
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  852 non-null    float64
 1   date_time             852 non-null    object 
 2   stage                 852 non-null    object 
 3   stadium               852 non-null    object 
 4   city                  852 non-null    object 
 5   home_team_name        852 non-null    object 
 6   home_team_goals       852 non-null    float64
 7   away_team_goals       852 non-null    float64
 8   away_team_name        852 non-null    object 
 9   win_conditions        852 non-null    object 
 10  attendance            850 non-null    float64
 11  half_time_home_goals  852 non-null    float64
 12  half_time_away_goals  852 non-null    float64
 13  referee               852 non-null    object 
 14  assistant_1           852 non-null    object 
 15  assistant_2          

In [9]:
WorldCups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   year             20 non-null     int64 
 1   country          20 non-null     object
 2   winner           20 non-null     object
 3   runners_up       20 non-null     object
 4   third            20 non-null     object
 5   fourth           20 non-null     object
 6   goals_scored     20 non-null     int64 
 7   qualified_teams  20 non-null     int64 
 8   matches_played   20 non-null     int64 
 9   attendance       20 non-null     object
dtypes: int64(4), object(6)
memory usage: 1.7+ KB


<a id="2.2.1"></a>
### Format date features

In [10]:
# Now let's create new features from date column

# Cast date columns to the Date data type
WorldCupMatches['date_time'] = pd.to_datetime(WorldCupMatches['date_time'])

WorldCupMatches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572 entries, 0 to 4571
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   year                  852 non-null    float64       
 1   date_time             852 non-null    datetime64[ns]
 2   stage                 852 non-null    object        
 3   stadium               852 non-null    object        
 4   city                  852 non-null    object        
 5   home_team_name        852 non-null    object        
 6   home_team_goals       852 non-null    float64       
 7   away_team_goals       852 non-null    float64       
 8   away_team_name        852 non-null    object        
 9   win_conditions        852 non-null    object        
 10  attendance            850 non-null    float64       
 11  half_time_home_goals  852 non-null    float64       
 12  half_time_away_goals  852 non-null    float64       
 13  referee           

# DROP NA & DUPLICATION

### + world_cup dataframe: year is the primary key

### + world_cup_match dataframe: year,match_id are the primary key

In [11]:
print("The WorldCupMatches dataframe BEFORE dropped has {} rows and {} columns".format(WorldCupMatches.shape[0], WorldCupMatches.shape[1]))
WorldCupMatches = WorldCupMatches.dropna(subset=["year","match_id"]).drop_duplicates(subset=["year", "match_id"], keep="first")
# display missing values in descending
print("The WorldCupMatches dataframe AFTER dropped has {} rows and {} columns".format(WorldCupMatches.shape[0], WorldCupMatches.shape[1]),"\n\n\n")


# WorldCups
print("The WorldCups dataframe BEFORE dropped has {} rows and {} columns".format(WorldCups.shape[0], WorldCups.shape[1]))
WorldCups = WorldCups.dropna(subset=["year"])
# display missing values in descending
print("The WorldCups dataframe AFTER dropped has {} rows and {} columns".format(WorldCups.shape[0], WorldCups.shape[1]),"\n\n\n")

The WorldCupMatches dataframe BEFORE dropped has 4572 rows and 20 columns
The WorldCupMatches dataframe AFTER dropped has 836 rows and 20 columns 



The WorldCups dataframe BEFORE dropped has 20 rows and 10 columns
The WorldCups dataframe AFTER dropped has 20 rows and 10 columns 





In [12]:
# print out list of District types
print(f'NUMBER OF CATEGORIES: {WorldCupMatches.win_conditions.nunique()}; \n\nUNIQUE NAMES OF THE CATEGORIES {WorldCupMatches.win_conditions.unique()}\n')

NUMBER OF CATEGORIES: 43; 

UNIQUE NAMES OF THE CATEGORIES [' ' 'Austria win after extra time ' 'Italy win after extra time '
 'Brazil win after extra time ' 'Czechoslovakia win after extra time '
 'Hungary win after extra time ' 'Northern Ireland win after extra time '
 'England win after extra time ' 'Germany FR win after extra time '
 'Uruguay win after extra time ' 'Argentina win after extra time '
 ' win on penalties (5 - 4) ' 'Belgium win after extra time '
 'France win on penalties (3 - 4) ' 'Germany FR win on penalties (4 - 1) '
 'Belgium win on penalties (4 - 5) ' 'France win after extra time '
 'Cameroon win after extra time '
 'Republic of Ireland win on penalties (5 - 4) '
 'Yugoslavia win after extra time ' ' win on penalties (2 - 3) '
 ' win on penalties (3 - 4) ' ' win on penalties (4 - 3) '
 'Bulgaria win on penalties (1 - 3) ' 'Sweden win on penalties (4 - 5) '
 'Brazil win on penalties (3 - 2) ' 'Argentina win on penalties (4 - 3) '
 'Brazil win on penalties (4 - 2) '

# Extract win_conditions column into multiple columns to show home team or away team wins

+ when_win_conditions: fulltime, extratime, penalty. 
+ penalty: Penalty Goals of Home Team and Away Team

In [13]:
import re
exp = re.compile(r'\((.*?)\)')
WorldCupMatches[['penalty']] = WorldCupMatches['win_conditions'].str.extract(exp, expand=True).fillna('')

when_win_conditions = "fulltime|extratime|penalty"
WorldCupMatches['when_win_conditions'] = WorldCupMatches.win_conditions.str.extract('('+when_win_conditions+')', expand=False)

WorldCupMatches.loc[WorldCupMatches['win_conditions'].str.contains("extra time"), 'when_win_conditions'] = 'extratime'
WorldCupMatches.loc[WorldCupMatches['win_conditions'].str.contains("penalties"), 'when_win_conditions'] = 'penalty'

In [14]:
# print out list of District types
print(f'NUMBER OF CATEGORIES: {WorldCupMatches.when_win_conditions.nunique()}; \n\nUNIQUE NAMES OF THE CATEGORIES {WorldCupMatches.when_win_conditions.unique()}\n')

NUMBER OF CATEGORIES: 2; 

UNIQUE NAMES OF THE CATEGORIES [nan 'extratime' 'penalty']



In [15]:
WorldCupMatches

,year,date_time,stage,stadium,city,home_team_name,home_team_goals,away_team_goals,away_team_name,win_conditions,...,half_time_away_goals,referee,assistant_1,assistant_2,round_id,match_id,home_team_initials,away_team_initials,penalty,when_win_conditions
0,1930.0,1930-07-13 15:00:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,...,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX,,NaN
1,1930.0,1930-07-13 15:00:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,...,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL,,NaN
2,1930.0,1930-07-14 12:45:00,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,...,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA,,NaN
3,1930.0,1930-07-14 14:50:00,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,...,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER,,NaN
4,1930.0,1930-07-15 16:00:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,...,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,2014.0,2014-07-05 13:00:00,Quarter-finals,Estadio Nacional,Brasilia,Argentina,1.0,0.0,Belgium,,...,0.0,Nicola RIZZOLI (ITA),Renato FAVERANI (ITA),Andrea STEFANI (ITA),255953.0,300186504.0,ARG,BEL,,NaN
832,2014.0,2014-06-29 13:00:00,Round of 16,Estadio Castelao,Fortaleza,Netherlands,2.0,1.0,Mexico,,...,0.0,PROENCA Pedro (POR),MIRANDA Bertino (POR),TRIGO Jose (POR),255951.0,300186508.0,NED,MEX,,NaN
833,2014.0,2014-06-29 17:00:00,Round of 16,Arena Pernambuco,Recife,Costa Rica,1.0,1.0,Greece,Costa Rica win on penalties (5 - 3),...,0.0,Ben WILLIAMS (AUS),CREAM Matthew (AUS),ANAZ Hakan (AUS),255951.0,300186459.0,CRC,GRE,5 - 3,penalty
834,2014.0,2014-07-01 13:00:00,Round of 16,Arena de Sao Paulo,Sao Paulo,Argentina,1.0,0.0,Switzerland,Argentina win after extra time,...,0.0,ERIKSSON Jonas (SWE),KLASENIUS Mathias (SWE),WARNMARK Daniel (SWE),255951.0,300186503.0,ARG,SUI,,extratime


<a id="3"></a>
<h1 style="color:#ffc0cb;font-size:40px;font-family:Georgia;text-align:center;"><strong>3. Save the Intermediate data that has been transformed</strong></h1>

In [16]:
# set the path of the cleaned data to data 
interim_data_path = os.path.join(os.path.pardir,'data','interim')
write_interim_path = os.path.join(interim_data_path, 'WorldCupMatches.csv')
# To write the data from the data frame into a file, use the to_csv function.
WorldCupMatches.to_csv(write_interim_path, index=False)
# df.to_csv('Dash/cleaned_data.csv', index=False)
print("cleaned WorldCupMatches data was successfully saved!")

cleaned WorldCupMatches data was successfully saved!


In [17]:
# set the path of the cleaned data to data 
write_interim_path = os.path.join(interim_data_path, 'WorldCups.csv')
# To write the data from the data frame into a file, use the to_csv function.
WorldCups.to_csv(write_interim_path, index=False)
print("cleaned WorldCups data was successfully saved!")

cleaned WorldCups data was successfully saved!
